# Load SOMC_AC

In [1]:
import numpy as np
import pandas as pd

In [19]:
# Load the CSV file
path = "../Raw Data/SOMC_AC_20240505_20250505_3600.csv"
df = pd.read_csv(path)

df.head(5)

,toll_booth,gantry,data_time,ac_a_current,ac_b_current,rs485_temperature,mains_power_220_current,mains_power_l1_voltage,mains_power_l2_voltage,smr_dc_current,smr_dc_voltage
0,泰山,01F-000.5N,2024-06-01 00:00:05,0.1,0.2,16.4,5.6,79.3,155.3,36.0,26.7
1,泰山,01F-000.5S,2024-06-01 00:00:05,0.1,2.0,22.1,7.1,115.0,122.4,35.3,26.7
2,泰山,01F-001.7S,2024-06-01 00:00:05,0.1,1.0,19.6,8.8,115.6,115.7,55.5,26.8
3,泰山,01F-002.9S,2024-06-01 00:00:05,0.1,0.2,21.4,8.7,117.1,117.2,57.7,26.8
4,泰山,01F-006.1S,2024-06-01 00:00:05,1.9,0.2,17.7,11.3,116.7,118.4,66.4,26.8


# Filters the DataFrame to return only rows where the toll_booth matches the given name

In [6]:
def get_toll_booth_data(df, toll_booth_name):
    return df[df['toll_booth'] == toll_booth_name]

In [20]:
taishan_df = get_toll_booth_data(df, "泰山")
taishan_df.tail(5)

,toll_booth,gantry,data_time,ac_a_current,ac_b_current,rs485_temperature,mains_power_220_current,mains_power_l1_voltage,mains_power_l2_voltage,smr_dc_current,smr_dc_voltage
1703394,泰山,01H-033.3N,2025-05-04 23:00:30,1.7,0.1,19.9,7.1,116.6,116.4,36.8,26.4
1703395,泰山,01H-033.4S,2025-05-04 23:00:30,1.6,0.1,20.6,7.2,115.9,115.9,36.8,26.4
1703396,泰山,01H-044.7N,2025-05-04 23:00:30,1.4,0.1,18.9,6.9,116.2,116.1,38.6,26.8
1703397,泰山,01H-044.7S,2025-05-04 23:00:30,0.2,0.1,22.8,6.0,114.5,114.4,38.9,26.7
1703399,泰山,01H-057.9S,2025-05-04 23:00:32,1.5,0.1,17.6,7.4,115.9,115.6,40.8,26.7


## Aggregate reading by Day

In [13]:
# Convert to datetime
df['data_time'] = pd.to_datetime(df['data_time'])

# Extract date
df['date'] = df['data_time'].dt.date

# Group by date (optional: add 'toll_booth' or 'gantry' for finer granularity)
daily_agg = df.groupby(['toll_booth', 'gantry', 'date']).agg({
    'ac_a_current': 'mean',
    'ac_b_current': 'mean',
    'rs485_temperature': 'mean',
    'mains_power_220_current': 'mean',
    'mains_power_l1_voltage': 'mean',
    'mains_power_l2_voltage': 'mean',
    'smr_dc_current': 'mean',
    'smr_dc_voltage': 'mean'
}).reset_index()

In [25]:
daily_agg.head(10)

,toll_booth,gantry,date,ac_a_current,ac_b_current,rs485_temperature,mains_power_220_current,mains_power_l1_voltage,mains_power_l2_voltage,smr_dc_current,smr_dc_voltage
0,后里,01F-104.5N,2024-06-01,0.100000,2.095833,18.820833,11.554167,114.704167,115.029167,66.220833,26.758333
1,后里,01F-104.5N,2024-06-02,1.608333,0.100000,18.158333,10.900000,116.066667,116.412500,66.137500,26.750000
2,后里,01F-104.5N,2024-06-03,1.579167,0.100000,18.308333,10.737500,116.391667,116.704167,66.120833,26.762500
3,后里,01F-104.5N,2024-06-04,0.204167,1.516667,17.987500,10.845833,114.833333,115.083333,65.708333,26.758333
4,后里,01F-104.5N,2024-06-05,1.604167,0.100000,17.987500,10.866667,114.891667,115.208333,65.408333,26.758333
5,后里,01F-104.5N,2024-06-06,0.100000,1.612500,17.629167,10.850000,115.295833,115.579167,65.804167,26.750000
6,后里,01F-104.5N,2024-06-07,1.687500,0.100000,18.308333,11.070833,115.004167,115.345833,66.116667,26.754167
7,后里,01F-104.5N,2024-06-08,0.100000,2.029167,17.795833,11.508333,114.270833,114.554167,66.591667,26.766667
8,后里,01F-104.5N,2024-06-09,1.854167,0.100000,18.612500,11.145833,116.245833,116.608333,66.204167,26.741667
9,后里,01F-104.5N,2024-06-10,2.158333,0.100000,19.195833,11.516667,114.854167,115.241667,66.308333,26.733333


# Get all toll booths 

In [27]:
unique_toll_booths = daily_agg['toll_booth'].unique()
print(unique_toll_booths)

['后里' '員林' '新市' '楊梅' '樹林' '泰山' '田寮']


# Toll Booths:

['后里' '員林' '新市' '楊梅' '樹林' '泰山' '田寮']

- 后里
- 員林
- 新市
- 楊梅
- 樹林
- 泰山
- 田寮

# Get The range of time on the Data Set

In [ ]:
# Convert 'date' column to datetime if it's not already
daily_agg['date'] = pd.to_datetime(daily_agg['date'])

# Get the min and max dates
start_date = daily_agg['date'].min()
end_date = daily_agg['date'].max()

print(f"Dataset covers from {start_date.date()} to {end_date.date()}")


Dataset covers from 2024-06-01 to 2025-05-04


# Export aggregated Reading as csv

In [ ]:
output_path = "../Clean Data/aggregated_day_SOMC_AC_20240505_20250505_3600.csv"
daily_agg.to_csv(output_path, index=False)

# Electricity Data of the toll station phase 1

In [38]:
path = "../Raw Data/Electricity of the toll station-phase 1 simplified.csv"

electricity_toll_station_df = pd.read_csv(path, skiprows=1)

electricity_toll_station_df.head(5)

,Location ID,Latitude,Longitude,Equipment Lane Count,Lane Count,Electricity Number,Billing Month,Electricity Consumption (kWh),Feeder Line Category,Power Outage Group,...,Lighting Usage Category,Number of Usage Days Last Year (Second Half),Billed kWh - Regular (Peak),Number of Usage Days Same Period Last Year,Average Electricity Consumption,Electricity Usage Same Period Last Year,Electricity Usage Last Year (Second Half),Number of Usage Days This Period,Unnamed: 21,Maintenance Station
0,01F0233N,25.073019,121.530703,5.0,4.0,00-03-9910-00-3,11307,2453,ZQ55,C,...,B21,60,2453,61,2453,2457,2472,59,01F-023.3N,北區泰山
1,01F0233N,25.073019,121.530703,5.0,4.0,00-03-9910-00-3,11309,2672,ZQ55,C,...,B21,61,2672,60,2672,2472,2375,63,01F-023.3N,北區泰山
2,01F0233N,25.073019,121.530703,5.0,4.0,00-03-9910-00-3,11311,2605,ZQ55,C,...,B21,63,2605,61,2605,2375,2573,62,01F-023.3N,北區泰山
3,01F0233N,25.073019,121.530703,5.0,4.0,00-03-9910-00-3,11401,2294,ZQ55,C,...,B21,61,2294,63,2294,2573,2418,58,01F-023.3N,北區泰山
4,01F0256N,25.078067,121.509106,6.0,4.0,00-21-9981-00-1,11305,2645,SA39,A,...,B21,61,2645,56,983,2213,2454,66,01F-025.6N,北區泰山


In [41]:
# Get all unique Maintenance Stations

unique_stations = electricity_toll_station_df['Maintenance Station'].unique()
print(unique_stations)

['北區泰山' '北區樹林' '北區楊梅' '行政部門' '中區后里' '中區員林' '南區新市' '南區田寮' 'IT機房']


# Normalize the Mantainance Station names

In [43]:
# Define allowed station names (as per your target list)
valid_stations = ['后里', '員林', '新市', '楊梅', '樹林', '泰山', '田寮']

# Function to extract matching station name
def clean_station_name(name):
    for station in valid_stations:
        if station in name:
            return station
    return None  # Filter out unwanted entries

# Apply the cleaning
electricity_toll_station_df['Station'] = electricity_toll_station_df['Maintenance Station'].apply(clean_station_name)

# Drop rows with None (i.e., unmatched stations like 行政部門, IT機房)
electricity_toll_station_df = electricity_toll_station_df.dropna(subset=['Station'])

# Optional: check unique cleaned names
print(electricity_toll_station_df['Station'].unique())


['泰山' '樹林' '楊梅' '后里' '員林' '新市' '田寮']


# Normalize Dates

In [49]:
# Function to convert Minguo date (e.g. 11307 or 1130624) to Gregorian YYYY-MM or YYYY-MM-DD
def convert_minguo_date(value):
    if pd.isna(value):
        return None
    value_str = str(int(value))
    if len(value_str) == 5:  # e.g., 11307 → YYYY-MM
        year = int(value_str[:3]) + 1911
        month = int(value_str[3:])
        return f"{year:04d}-{month:02d}"
    elif len(value_str) == 7:  # e.g., 1130624 → YYYY-MM-DD
        year = int(value_str[:3]) + 1911
        month = int(value_str[3:5])
        day = int(value_str[5:])
        return f"{year:04d}-{month:02d}-{day:02d}"
    else:
        return None

In [50]:
# Columns with Minguo dates
minguo_date_columns = [
    'Billing Month', 'Billing Period (Start)', 'Billing Period (End)', 'Billing Date'
]

# Apply conversion
for col in minguo_date_columns:
    electricity_toll_station_df[col] = electricity_toll_station_df[col].apply(convert_minguo_date)

# Preview result
print(electricity_toll_station_df[[col for col in minguo_date_columns]].head())

  Billing Month Billing Period (Start) Billing Period (End) Billing Date
0       2024-07             2024-04-29           2024-06-26   2024-07-03
1       2024-09             2024-06-27           2024-08-28   2024-09-04
2       2024-11             2024-08-29           2024-10-29   2024-11-05
3       2025-01             2024-10-30           2024-12-26   2025-01-06
4       2024-05             2024-02-27           2024-05-02   2024-05-09


In [51]:
electricity_toll_station_df.head()

,Location ID,Latitude,Longitude,Equipment Lane Count,Lane Count,Electricity Number,Billing Month,Electricity Consumption (kWh),Feeder Line Category,Power Outage Group,...,Number of Usage Days Last Year (Second Half),Billed kWh - Regular (Peak),Number of Usage Days Same Period Last Year,Average Electricity Consumption,Electricity Usage Same Period Last Year,Electricity Usage Last Year (Second Half),Number of Usage Days This Period,Unnamed: 21,Maintenance Station,Station
0,01F0233N,25.073019,121.530703,5.0,4.0,00-03-9910-00-3,2024-07,2453,ZQ55,C,...,60,2453,61,2453,2457,2472,59,01F-023.3N,北區泰山,泰山
1,01F0233N,25.073019,121.530703,5.0,4.0,00-03-9910-00-3,2024-09,2672,ZQ55,C,...,61,2672,60,2672,2472,2375,63,01F-023.3N,北區泰山,泰山
2,01F0233N,25.073019,121.530703,5.0,4.0,00-03-9910-00-3,2024-11,2605,ZQ55,C,...,63,2605,61,2605,2375,2573,62,01F-023.3N,北區泰山,泰山
3,01F0233N,25.073019,121.530703,5.0,4.0,00-03-9910-00-3,2025-01,2294,ZQ55,C,...,61,2294,63,2294,2573,2418,58,01F-023.3N,北區泰山,泰山
4,01F0256N,25.078067,121.509106,6.0,4.0,00-21-9981-00-1,2024-05,2645,SA39,A,...,61,2645,56,983,2213,2454,66,01F-025.6N,北區泰山,泰山


# Export Electricity Data of toll 

In [ ]:
output_path = "../Clean Data/aggregated_day_SOMC_AC_20240505_20250505_3600.csv"
electricity_toll_station_df.to_csv(output_path, index=False)